In [ ]:
POP_SIZE = 100
MUTATION_RATE = 0.3
CROSSOVER_RATE_PARENT = 0.8
TOURNAMENT_SIZE = 5
MAX_GENERATIONS = 20000

In [ ]:
import time
import numpy as np
import random
import itertools

easy1_sudoku = np.array([
    [0, 4, 0, 0, 3, 8, 0, 0, 7],
    [0, 7, 9, 0, 1, 0, 0, 0, 8],
    [0, 5, 8, 0, 0, 0, 0, 2, 0],
    [0, 6, 4, 0, 0, 5, 0, 0, 3],
    [0, 0, 7, 3, 0, 4, 8, 0, 0],
    [2, 0, 0, 7, 0, 0, 5, 6, 0],
    [0, 2, 0, 0, 0, 0, 7, 3, 0],
    [7, 0, 0, 0, 5, 0, 6, 4, 0],
    [4, 0, 0, 1, 9, 0, 0, 8, 5]
])

medium1_sudoku = np.array([
    [6, 0, 0, 0, 3, 0, 0, 0, 1],
    [0, 1, 0, 6, 9, 0, 2, 8, 0],
    [5, 0, 9, 0, 0, 0, 0, 0, 0],
    [0, 6, 2, 0, 8, 3, 0, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 0, 2, 7, 0, 3, 1, 0],
    [0, 0, 0, 0, 0, 0, 5, 0, 2],
    [0, 5, 4, 0, 6, 7, 0, 9, 0],
    [9, 0, 0, 0, 5, 0, 0, 0, 8]
])

hard1_sudoku = np.array([
    [9, 5, 6, 3, 0, 1, 8, 0, 0],
    [0, 0, 0, 0, 4, 0, 2, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 9],
    [0, 6, 0, 4, 0, 0, 5, 0, 0],
    [4, 0, 0, 0, 6, 0, 0, 0, 7],
    [0, 0, 1, 0, 0, 2, 0, 6, 0],
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 9, 0, 0, 0, 0],
    [0, 0, 2, 7, 0, 4, 9, 3, 6],
])

# ------------------------------
# Pretty print Sudoku grid
# ------------------------------
def print_sudoku(grid, title=None):
    if title:
        print("\n" + "=" * 40)
        print(title)
        print("=" * 40)
    for i, row in enumerate(grid):
        row_str = ""
        for j, val in enumerate(row):
            char = "." if val == 0 else str(val)
            sep = " "
            if j in [2, 5]:
                sep = " | "
            row_str += char + sep
        print(row_str)
        if i in [2, 5]:
            print("-" * 21)
    print("=" * 40)

# ------------------------------
# Precompute candidate map (dari GIVENS saja)
# ------------------------------
def precompute_candidate_map(base_grid):
    fixed = base_grid != 0
    cand = {}
    for r in range(9):
        row_used = set(base_grid[r, fixed[r, :]])
        for c in range(9):
            if fixed[r, c]:
                continue
            col_used = set(base_grid[fixed[:, c], c])
            br, bc = (r // 3) * 3, (c // 3) * 3
            block = base_grid[br:br+3, bc:bc+3]
            block_fixed = fixed[br:br+3, bc:bc+3]
            block_used = set(block[block_fixed])
            used = row_used | col_used | block_used
            cand[(r, c)] = set(range(1, 10)) - used
    return cand, fixed

# ------------------------------
# Posisi non-given per blok 3x3
# ------------------------------
def compute_block_positions(base_grid, fixed):
    block_pos = {}
    for bx in range(3):
        for by in range(3):
            rows = range(bx*3, (bx+1)*3)
            cols = range(by*3, (by+1)*3)
            positions = []
            for r in rows:
                for c in cols:
                    if not fixed[r, c]:
                        positions.append((r, c))
            block_pos[(bx, by)] = positions
    return block_pos

# ------------------------------
# Individual: hanya angka non-given per blok
# ------------------------------
def create_individual(base_grid, fixed, block_pos):
    ind = {}
    for (bx, by), positions in block_pos.items():
        br, bc = bx*3, by*3
        block = base_grid[br:br+3, bc:bc+3]
        block_fixed_mask = fixed[br:br+3, bc:bc+3]
        used = set(block[block_fixed_mask])
        missing = list(set(range(1, 10)) - used)         # pastikan blok valid (1..9)
        random.shuffle(missing)
        # len(missing) == jumlah sel non-given pada blok
        ind[(bx, by)] = missing[:len(positions)]
    return ind

# ------------------------------
# Decode: isi non-given ke grid copy
# ------------------------------
def decode_to_grid(base_grid, individual, block_pos):
    grid = base_grid.copy()
    for (bx, by), positions in block_pos.items():
        vals = individual[(bx, by)]
        for i, (r, c) in enumerate(positions):
            grid[r, c] = vals[i]
    return grid

# ------------------------------
# Fitness: duplikasi baris + kolom
# (blok valid by construction)
# ------------------------------
def fitness_individual(individual, base_grid, block_pos):
    # buat set berisi nilai unik per baris dan kolom
    row_vals = [set() for _ in range(9)]
    col_vals = [set() for _ in range(9)]

    # tambahkan semua angka 'given' dulu
    fixed = base_grid != 0
    for r in range(9):
        for c in range(9):
            if fixed[r, c]:
                val = base_grid[r, c]
                row_vals[r].add(val)
                col_vals[c].add(val)

    # tambahkan semua angka dari individu
    for (bx, by), positions in block_pos.items():
        vals = individual[(bx, by)]
        for i, (r, c) in enumerate(positions):
            v = vals[i]
            row_vals[r].add(v)
            col_vals[c].add(v)

    # hitung penalti duplikasi
    score = 0
    for i in range(9):
        score += (9 - len(row_vals[i]))  # baris
        score += (9 - len(col_vals[i]))  # kolom

    return score


# ------------------------------
# Tournament selection
# ------------------------------
def tournament_selection(population, base_grid, block_pos, k=TOURNAMENT_SIZE):
    selected = random.sample(population, k)
    selected.sort(key=lambda ind: fitness_individual(ind, base_grid, block_pos))
    return selected[0]

# ------------------------------
# Crossover: block-wise
# ------------------------------

def crossover1(p1, p2):
    child = {}
    for bx in range(3):
        for by in range(3):
            key = (bx, by)
            # ambil blok dari salah satu parent
            child[key] = list(p1[key]) if random.random() < 0.5 else list(p2[key])
    return child

def crossover2(p1, p2, base_grid, block_pos):
    """
    Versi efisien tanpa decode_to_grid.
    Menilai skor baris dan kolom langsung dari genotipe per-blok.
    """
    # precompute: isi baris dan kolom untuk kedua parent
    fixed = base_grid != 0
    row_vals_p1 = [set(base_grid[r, fixed[r, :]]) for r in range(9)]
    col_vals_p1 = [set(base_grid[fixed[:, c], c]) for c in range(9)]
    row_vals_p2 = [set(rv) for rv in row_vals_p1]
    col_vals_p2 = [set(cv) for cv in col_vals_p1]

    for (bx, by), positions in block_pos.items():
        vals1 = p1[(bx, by)]
        vals2 = p2[(bx, by)]
        for i, (r, c) in enumerate(positions):
            row_vals_p1[r].add(vals1[i])
            col_vals_p1[c].add(vals1[i])
            row_vals_p2[r].add(vals2[i])
            col_vals_p2[c].add(vals2[i])

    def row_score(row_vals):  # total unique per 3 baris
        return sum(len(row_vals[r]) for r in rows)

    def col_score(col_vals):
        return sum(len(col_vals[c]) for c in cols)

    # hasil child berdasarkan row dan col
    child_row, child_col = {}, {}

    for bx in range(3):
        for by in range(3):
            rows = range(bx*3, (bx+1)*3)
            cols = range(by*3, (by+1)*3)

            sum_r_p1 = sum(len(row_vals_p1[r]) for r in rows)
            sum_r_p2 = sum(len(row_vals_p2[r]) for r in rows)
            sum_c_p1 = sum(len(col_vals_p1[c]) for c in cols)
            sum_c_p2 = sum(len(col_vals_p2[c]) for c in cols)

            child_row[(bx, by)] = list(p2[(bx, by)]) if sum_r_p2 > sum_r_p1 else list(p1[(bx, by)])
            child_col[(bx, by)] = list(p2[(bx, by)]) if sum_c_p2 > sum_c_p1 else list(p1[(bx, by)])

    fit_r = fitness_individual(child_row, base_grid, block_pos)
    fit_c = fitness_individual(child_col, base_grid, block_pos)
    return child_row if fit_r <= fit_c else child_col


# ------------------------------
# Filtered mutation (pakai candidate_map)
# swap 2 sel non-given DI DALAM blok jika lolos filter
# ------------------------------
def mutate(individual, mutation_rate, candidate_map, block_pos):
    new_ind = {k: list(v) for k, v in individual.items()}
    for (bx, by), positions in block_pos.items():
        if len(positions) < 2:
            continue
        if random.random() >= mutation_rate:
            continue

        pairs = list(itertools.combinations(range(len(positions)), 2))
        random.shuffle(pairs)

        for i, j in pairs:
            (r1, c1) = positions[i]
            (r2, c2) = positions[j]
            v1 = new_ind[(bx, by)][i]
            v2 = new_ind[(bx, by)][j]
            cand1 = candidate_map.get((r1, c1), set())
            cand2 = candidate_map.get((r2, c2), set())
            if (v1 in cand2) or (v2 in cand1):
                new_ind[(bx, by)][i], new_ind[(bx, by)][j] = v2, v1
                break
    return new_ind


# ------------------------------
# GA main loop
# ------------------------------
def genetic_sudoku_solver(base_grid, crossover_func):
    candidate_map, fixed = precompute_candidate_map(base_grid)
    block_pos = compute_block_positions(base_grid, fixed)
    population = [create_individual(base_grid, fixed, block_pos) for _ in range(POP_SIZE)]

    sample_ind = population[0]
    print("\n===== Contoh Individu Awal (GENOTYPE) =====")
    for key, val in sample_ind.items():
        print(f"Blok {key}: {val}")

    # 🔹 Decode dan print hasil grid-nya
    sample_grid = decode_to_grid(base_grid, sample_ind, block_pos)
    print_sudoku(sample_grid, "Contoh Individu Awal (PHENOTYPE)")

    # ⏱️ Mulai proses evolusi
    print("\nMulai evolusi...")
    total_start = time.time()

    for gen in range(MAX_GENERATIONS):
        gen_start = time.time()  # waktu mulai generasi

        population.sort(key=lambda ind: fitness_individual(ind, base_grid, block_pos))
        best = population[0]
        best_fit = fitness_individual(best, base_grid, block_pos)

        gen_time = time.time() - gen_start  # durasi generasi ini

        if gen % 10 == 0:
            print(f"Gen {gen:4d} | Fitness terbaik = {best_fit:2d} | Waktu per generasi = {gen_time:.4f} detik")

        if best_fit == 0:
            total_time = time.time() - total_start
            print(f"\n✅ Solusi ditemukan di generasi {gen} fitness: {best_fit}")
            print(f"Total waktu eksekusi: {total_time:.4f} detik")
            return decode_to_grid(base_grid, best, block_pos)

        new_population = []
        for _ in range(POP_SIZE):
            p1 = tournament_selection(population, base_grid, block_pos)
            p2 = tournament_selection(population, base_grid, block_pos)
            if random.random() < CROSSOVER_RATE_PARENT:
                child = crossover_func(p1, p2)
            else:
                # tanpa crossover: copy parent yang lebih fit
                child = p1 if fitness_individual(p1, base_grid, block_pos) < fitness_individual(p2, base_grid, block_pos) else p2
                child = {k: list(v) for k, v in child.items()}
            child = mutate(child, MUTATION_RATE, candidate_map, block_pos)
            new_population.append(child)

        population = new_population

    total_time = time.time() - total_start
    print("\n❌ Tidak ditemukan solusi dalam batas generasi.")
    print(f"Total waktu eksekusi: {total_time:.4f} detik")
    return decode_to_grid(base_grid, best, block_pos)


# ------------------------------
# Jalankan
# ------------------------------
# print_sudoku(sudoku, "Puzzle Awal")
# solution = genetic_sudoku_solver(sudoku)
# print_sudoku(solution, "Solusi Akhir (GA)")

In [ ]:
def run_experiment(seed, crossover_func, sudoku_grid, label=""):
    random.seed(seed)
    np.random.seed(seed)

    print(f"\n=== Eksperimen Seed {seed} | Puzzle {label or 'Custom'} | "
          f"Crossover = {crossover_func.__name__} ===")

    solution = genetic_sudoku_solver(sudoku_grid, crossover_func=crossover_func)
    print_sudoku(solution, f"Solusi Akhir (Seed {seed} - {label or 'Custom'})")


In [ ]:
SEED = 432      # ubah angka ini untuk eksperimen lain
SUDOKU = hard1_sudoku

In [ ]:
run_experiment(SEED, crossover1, easy1_sudoku)


=== Eksperimen Seed 432 | Puzzle Custom | Crossover = crossover1 ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [3, 2, 1, 6]
Blok (0, 1): [9, 2, 5, 7, 4, 6]
Blok (0, 2): [3, 4, 6, 1, 5, 9]
Blok (1, 0): [1, 3, 8, 9, 5]
Blok (1, 1): [1, 8, 6, 9, 2]
Blok (1, 2): [4, 7, 9, 1, 2]
Blok (2, 0): [3, 1, 6, 5, 8, 9]
Blok (2, 1): [4, 8, 3, 7, 2, 6]
Blok (2, 2): [1, 9, 2]

Contoh Individu Awal (PHENOTYPE)
3 4 2 | 9 3 8 | 3 4 7 
1 7 9 | 2 1 5 | 6 1 8 
6 5 8 | 7 4 6 | 5 2 9 
---------------------
1 6 4 | 1 8 5 | 4 7 3 
3 8 7 | 3 6 4 | 8 9 1 
2 9 5 | 7 9 2 | 5 6 2 
---------------------
3 2 1 | 4 8 3 | 7 3 1 
7 6 5 | 7 5 2 | 6 4 9 
4 8 9 | 1 9 6 | 2 8 5 

Mulai evolusi...
Gen    0 | Fitness terbaik = 30 | Waktu per generasi = 0.0048 detik
Gen   10 | Fitness terbaik = 12 | Waktu per generasi = 0.0038 detik
Gen   20 | Fitness terbaik = 10 | Waktu per generasi = 0.0036 detik
Gen   30 | Fitness terbaik =  8 | Waktu per generasi = 0.0036 detik
Gen   40 | Fitness terbaik =  8 | Waktu per ge

In [ ]:
run_experiment(SEED, crossover1, medium1_sudoku)


=== Eksperimen Seed 432 | Puzzle Custom | Crossover = crossover1 ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [4, 3, 2, 7, 8]
Blok (0, 1): [8, 1, 4, 7, 2, 5]
Blok (0, 2): [4, 5, 7, 3, 6, 9]
Blok (1, 0): [3, 8, 1, 5, 9, 4]
Blok (1, 1): [5, 9, 1, 4, 6]
Blok (1, 2): [5, 9, 7, 6, 8, 2]
Blok (2, 0): [3, 8, 2, 7, 1, 6]
Blok (2, 1): [2, 1, 8, 4, 9, 3]
Blok (2, 2): [1, 3, 6, 7, 4]

Contoh Individu Awal (PHENOTYPE)
6 4 3 | 8 3 1 | 4 5 1 
2 1 7 | 6 9 4 | 2 8 7 
5 8 9 | 7 2 5 | 3 6 9 
---------------------
3 6 2 | 5 8 3 | 5 9 7 
7 8 1 | 9 1 4 | 6 8 4 
5 9 4 | 2 7 6 | 3 1 2 
---------------------
3 8 2 | 2 1 8 | 5 1 2 
7 5 4 | 4 6 7 | 3 9 6 
9 1 6 | 9 5 3 | 7 4 8 

Mulai evolusi...
Gen    0 | Fitness terbaik = 35 | Waktu per generasi = 0.0041 detik
Gen   10 | Fitness terbaik = 17 | Waktu per generasi = 0.0044 detik
Gen   20 | Fitness terbaik = 12 | Waktu per generasi = 0.0039 detik
Gen   30 | Fitness terbaik =  8 | Waktu per generasi = 0.0044 detik
Gen   40 | Fitness terbaik = 10

In [ ]:
run_experiment(SEED, crossover1, SUDOKU)


=== Eksperimen Seed 432 | Puzzle Custom | Crossover = crossover1 ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [8, 3, 2, 1, 4, 7]
Blok (0, 1): [7, 2, 6, 8, 5, 9]
Blok (0, 2): [3, 4, 6, 1, 5, 7]
Blok (1, 0): [3, 8, 2, 7, 9, 5]
Blok (1, 1): [5, 8, 1, 3, 7, 9]
Blok (1, 2): [2, 9, 4, 3, 8, 1]
Blok (2, 0): [4, 9, 3, 6, 1, 5]
Blok (2, 1): [2, 1, 6, 5, 8, 3]
Blok (2, 2): [1, 2, 7, 5, 8, 4]

Contoh Individu Awal (PHENOTYPE)
9 5 6 | 3 7 1 | 8 3 4 
8 3 2 | 2 4 6 | 2 6 1 
1 4 7 | 8 5 9 | 5 7 9 
---------------------
3 6 8 | 4 5 8 | 5 2 9 
4 2 7 | 1 6 3 | 4 3 7 
9 5 1 | 7 9 2 | 8 6 1 
---------------------
8 4 9 | 2 1 6 | 1 2 7 
3 6 7 | 5 9 8 | 5 8 4 
1 5 2 | 7 3 4 | 9 3 6 

Mulai evolusi...
Gen    0 | Fitness terbaik = 28 | Waktu per generasi = 0.0055 detik
Gen   10 | Fitness terbaik = 19 | Waktu per generasi = 0.0039 detik
Gen   20 | Fitness terbaik = 14 | Waktu per generasi = 0.0035 detik
Gen   30 | Fitness terbaik = 12 | Waktu per generasi = 0.0039 detik
Gen   40 | Fitness ter

In [ ]:
block_pos_easy = compute_block_positions(easy1_sudoku, easy1_sudoku != 0)

run_experiment(
    SEED,
    # Bungkus pakai lambda biar solver taunya cuma (p1, p2)
    lambda p1, p2: crossover2(p1, p2, easy1_sudoku, block_pos_easy),
    easy1_sudoku
)


=== Eksperimen Seed 432 | Puzzle Custom | Crossover = <lambda> ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [3, 2, 1, 6]
Blok (0, 1): [9, 2, 5, 7, 4, 6]
Blok (0, 2): [3, 4, 6, 1, 5, 9]
Blok (1, 0): [1, 3, 8, 9, 5]
Blok (1, 1): [1, 8, 6, 9, 2]
Blok (1, 2): [4, 7, 9, 1, 2]
Blok (2, 0): [3, 1, 6, 5, 8, 9]
Blok (2, 1): [4, 8, 3, 7, 2, 6]
Blok (2, 2): [1, 9, 2]

Contoh Individu Awal (PHENOTYPE)
3 4 2 | 9 3 8 | 3 4 7 
1 7 9 | 2 1 5 | 6 1 8 
6 5 8 | 7 4 6 | 5 2 9 
---------------------
1 6 4 | 1 8 5 | 4 7 3 
3 8 7 | 3 6 4 | 8 9 1 
2 9 5 | 7 9 2 | 5 6 2 
---------------------
3 2 1 | 4 8 3 | 7 3 1 
7 6 5 | 7 5 2 | 6 4 9 
4 8 9 | 1 9 6 | 2 8 5 

Mulai evolusi...
Gen    0 | Fitness terbaik = 30 | Waktu per generasi = 0.0041 detik
Gen   10 | Fitness terbaik =  9 | Waktu per generasi = 0.0035 detik
Gen   20 | Fitness terbaik =  4 | Waktu per generasi = 0.0036 detik
Gen   30 | Fitness terbaik =  2 | Waktu per generasi = 0.0036 detik
Gen   40 | Fitness terbaik =  2 | Waktu per gene

In [ ]:
block_pos_easy = compute_block_positions(medium1_sudoku, medium1_sudoku != 0)

run_experiment(
    SEED,
    # Bungkus pakai lambda biar solver taunya cuma (p1, p2)
    lambda p1, p2: crossover2(p1, p2, medium1_sudoku, block_pos_easy),
    medium1_sudoku
)


=== Eksperimen Seed 432 | Puzzle Custom | Crossover = <lambda> ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [4, 3, 2, 7, 8]
Blok (0, 1): [8, 1, 4, 7, 2, 5]
Blok (0, 2): [4, 5, 7, 3, 6, 9]
Blok (1, 0): [3, 8, 1, 5, 9, 4]
Blok (1, 1): [5, 9, 1, 4, 6]
Blok (1, 2): [5, 9, 7, 6, 8, 2]
Blok (2, 0): [3, 8, 2, 7, 1, 6]
Blok (2, 1): [2, 1, 8, 4, 9, 3]
Blok (2, 2): [1, 3, 6, 7, 4]

Contoh Individu Awal (PHENOTYPE)
6 4 3 | 8 3 1 | 4 5 1 
2 1 7 | 6 9 4 | 2 8 7 
5 8 9 | 7 2 5 | 3 6 9 
---------------------
3 6 2 | 5 8 3 | 5 9 7 
7 8 1 | 9 1 4 | 6 8 4 
5 9 4 | 2 7 6 | 3 1 2 
---------------------
3 8 2 | 2 1 8 | 5 1 2 
7 5 4 | 4 6 7 | 3 9 6 
9 1 6 | 9 5 3 | 7 4 8 

Mulai evolusi...
Gen    0 | Fitness terbaik = 35 | Waktu per generasi = 0.0041 detik
Gen   10 | Fitness terbaik = 12 | Waktu per generasi = 0.0036 detik
Gen   20 | Fitness terbaik =  7 | Waktu per generasi = 0.0040 detik
Gen   30 | Fitness terbaik =  6 | Waktu per generasi = 0.0036 detik
Gen   40 | Fitness terbaik =  4 |

In [ ]:
block_pos_easy = compute_block_positions(hard1_sudoku, hard1_sudoku != 0)

run_experiment(
    SEED,
    # Bungkus pakai lambda biar solver taunya cuma (p1, p2)
    lambda p1, p2: crossover2(p1, p2, hard1_sudoku, block_pos_easy),
    hard1_sudoku
)


=== Eksperimen Seed 432 | Puzzle Custom | Crossover = <lambda> ===

===== Contoh Individu Awal (GENOTYPE) =====
Blok (0, 0): [8, 3, 2, 1, 4, 7]
Blok (0, 1): [7, 2, 6, 8, 5, 9]
Blok (0, 2): [3, 4, 6, 1, 5, 7]
Blok (1, 0): [3, 8, 2, 7, 9, 5]
Blok (1, 1): [5, 8, 1, 3, 7, 9]
Blok (1, 2): [2, 9, 4, 3, 8, 1]
Blok (2, 0): [4, 9, 3, 6, 1, 5]
Blok (2, 1): [2, 1, 6, 5, 8, 3]
Blok (2, 2): [1, 2, 7, 5, 8, 4]

Contoh Individu Awal (PHENOTYPE)
9 5 6 | 3 7 1 | 8 3 4 
8 3 2 | 2 4 6 | 2 6 1 
1 4 7 | 8 5 9 | 5 7 9 
---------------------
3 6 8 | 4 5 8 | 5 2 9 
4 2 7 | 1 6 3 | 4 3 7 
9 5 1 | 7 9 2 | 8 6 1 
---------------------
8 4 9 | 2 1 6 | 1 2 7 
3 6 7 | 5 9 8 | 5 8 4 
1 5 2 | 7 3 4 | 9 3 6 

Mulai evolusi...
Gen    0 | Fitness terbaik = 28 | Waktu per generasi = 0.0042 detik
Gen   10 | Fitness terbaik = 16 | Waktu per generasi = 0.0036 detik
Gen   20 | Fitness terbaik = 10 | Waktu per generasi = 0.0035 detik
Gen   30 | Fitness terbaik =  9 | Waktu per generasi = 0.0035 detik
Gen   40 | Fitness terba